In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

<h1 style="background-color:Skyblue;font-family:Georgia;font-size:350%;text-align:center"> Titanic Data EDA and Logestic Regression </h1>


<img src="https://data.whicdn.com/images/172028045/original.gif">





<h1 style="background-color:Skyblue;font-family:Georgia;font-size:250%;text-align:center">Analysis of the different features of the Titanic Data Set and Classification </h1>

<h1 style="background-color:BLUE;font-family:Georgia;font-size:100%;text-align:center">Please Upvote and comment if you like the EDA and classification</h1>


<h1 style="background-color:Skyblue;font-family:Georgia;font-size:175%;text-align:left"> Step 1: Importing  Data</h1>

In [ ]:
# Importing  datasets
train=pd.read_csv("/kaggle/input/titanic/train.csv")
train.head()

<h1 style="background-color:Skyblue;font-family:Georgia;font-size:175%;text-align:left">Step 2: Inspecting the Dataframe </h1>

In [ ]:
#to see the rows and columns
train.shape

In [ ]:
#to see the info 
train.info()

In [ ]:
#to see the 
train.describe()

<h1 style="background-color:Skyblue;font-family:Georgia;font-size:175%;text-align:left">Step 3: Data Cleaning and Preparation</h1>

In [ ]:
train = train.drop('Cabin', axis=1)

In [ ]:
train.isnull().sum()

In [ ]:
#taking only not null values from 'Age' and 'Embarked' since they have not null values
train=train[train['Age'].notnull()]
train=train[train['Embarked'].notnull()]

In [ ]:
train.isnull().sum()

<h1 style="background-color:Skyblue;font-family:Georgia;font-size:175%;text-align:left">Step 4: Data Visualization</h1>

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns


def plotcountgraph(var):
    
    sns.set(font_scale=1.25)
    plt.figure(figsize=(10, 5))
    ax = sns.countplot(x=var, hue='Survived', data=train,  palette="viridis")
    plt.setp(ax.get_xticklabels(), rotation=0)

In [ ]:
var=['Sex','Pclass','Embarked','SibSp','Parch']
for i in var:
    plotcountgraph(i)

<h1 style="background-color:Skyblue;font-family:Georgia;font-size:175%;text-align:left"> Inference from the above graphs
   
    
    *Number of females survived is more when compared to male
    *Lower the Pclass higher is the rate of survival
    *Survival rate of people embarked from Southampton is higher 
    *Survival rate of people having no siblings is higher
    *Survival rate of people having no parents / children aboard the Titanic is higher


In [ ]:
#Finding the correlation between the numeric variables
plt.figure(figsize = (20,10))
sns.heatmap(train.corr(),annot = True)
plt.show()

In [ ]:
#drpping the Parch as it is having low correlation with survived(target variable)
train = train.drop('Parch', axis=1)
#dropping ticket and Parch as it is not required for further analysis
train = train.drop('Ticket', axis=1)
train = train.drop('Name', axis=1)

In [ ]:
#importing the test data set and checking
test=pd.read_csv("/kaggle/input/titanic/test.csv")
test.head()

In [ ]:
#converting  the categorical variables to numeric as required for model building

dummy1 = pd.get_dummies(train[['Sex','Embarked']], drop_first=True)
# Adding the results to the master dataframe
train = pd.concat([train, dummy1], axis=1)

dummy1 = pd.get_dummies(test[['Sex','Embarked']], drop_first=True)
# Adding the results to the master dataframe
test = pd.concat([test, dummy1], axis=1)


In [ ]:
#since the dummies are already present
train=train.drop(['Sex','Embarked'],1)
test=test.drop(['Sex','Embarked'],1)


In [ ]:
train.head()

<h1 style="background-color:Skyblue;font-family:Georgia;font-size:175%;text-align:left">Step 5: Creating the Test and Train Data properly</h1>

In [ ]:
#dropping the target variable for training the model
X_train=train.drop('Survived', axis=1)


In [ ]:
#dataset for the target variable for training the model
y_train=train['Survived']

In [ ]:
#Creating the test data set X_test 
X_test=test


<h1 style="background-color:Skyblue;font-family:Georgia;font-size:175%;text-align:left">Step 6: Model Building</h1>

In [ ]:
#Running Your First Training Model
import statsmodels.api as sm
logm1 = sm.GLM(y_train,(sm.add_constant(X_train)), family = sm.families.Binomial())
logm1.fit().summary()

<h1 style="background-color:Skyblue;font-family:Georgia;font-size:175%;text-align:left">Step 7: Feature Selection Using RFE</h1>

In [ ]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()

In [ ]:
from sklearn.feature_selection import RFE
rfe = RFE(logreg,6)             # running RFE with 13 variables as output
rfe = rfe.fit(X_train, y_train)

In [ ]:
rfe.support_

In [ ]:
list(zip(X_train.columns, rfe.support_, rfe.ranking_))

In [ ]:
col = X_train.columns[rfe.support_]
X_train.columns[~rfe.support_]

Assessing the model with StatsModels

In [ ]:
X_train_sm = sm.add_constant(X_train[col])
logm2 = sm.GLM(y_train,X_train_sm, family = sm.families.Binomial())
res = logm2.fit()
res.summary()

In [ ]:
# Getting the predicted values on the train set
y_train_pred = res.predict(X_train_sm)
y_train_pred[:10]

In [ ]:
y_train_pred = y_train_pred.values.reshape(-1)
y_train_pred[:10]

 Creating a dataframe with the actual churn flag and the predicted probabilities

In [ ]:
y_train_pred_final = pd.DataFrame({'Survived':y_train.values, 'Survived_prob':y_train_pred})
y_train_pred_final['PassengerId'] = y_train.index
y_train_pred_final.head()

Creating new column 'predicted' with 1 if Survived_Prob > 0.5 else 0

In [ ]:
y_train_pred_final['predicted'] = y_train_pred_final.Survived_prob.map(lambda x: 1 if x > 0.54 else 0)

# Let's see the head
y_train_pred_final

In [ ]:
from sklearn import metrics
# Confusion matrix 
confusion = metrics.confusion_matrix(y_train_pred_final.Survived, y_train_pred_final.predicted )
print(confusion)

In [ ]:
# Let's check the overall accuracy.
print(metrics.accuracy_score(y_train_pred_final.Survived, y_train_pred_final.predicted))

### The accuracy is pretty Good!

<h1 style="background-color:Skyblue;font-family:Georgia;font-size:175%;text-align:left">Step 8: Making predictions on the test set</h1>

In [ ]:
X_test = X_test[col]
X_test.head()

In [ ]:
X_test_sm = sm.add_constant(X_test)

In [ ]:
#Making predictions on the test set
y_test_pred = res.predict(X_test_sm)

In [ ]:
y_test_pred[:10]

In [ ]:
## Converting y_pred to a dataframe which is an array
y_pred_1 = pd.DataFrame(y_test_pred)
y_pred_1.head()

In [ ]:
# Removing index for both dataframes to append them side by side 
y_pred_1.reset_index(drop=True, inplace=True)
#y_test_df.reset_index(drop=True, inplace=True)

In [ ]:
# creating a final 
y_pred_final = pd.concat([ y_pred_1],axis=1)

In [ ]:
y_pred_final.head()

In [ ]:
# Renaming and Reaaranging the column 
y_pred_final= y_pred_final.rename(columns={ 0 : 'Survived_Prob'})

In [ ]:
# Let's see the head of y_pred_final
y_pred_final.head()

In [ ]:
y_pred_final['Survived'] = y_pred_final.Survived_Prob.map(lambda x: 1 if x > 0.54 else 0)

In [ ]:
y_pred_final.head()

In [ ]:
#creating the submission.csv for the submission 

output = pd.DataFrame({'PassengerId': test.PassengerId , 'Survived': y_pred_final.Survived})
output.to_csv('my_submission.csv', index=False)
print("Your submission was successfully saved!")

In [ ]:
output = pd.DataFrame({'PassengerId': test.PassengerId , 'Survived': y_pred_final.Survived_Prob})
output.to_csv('my_submission_Prob.csv', index=False)